In [ ]:
# Load the dataset from Github repository
!git clone https://github.com/procodingclass/product_dataset.git

Cloning into 'product_dataset'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 3.08 MiB | 13.52 MiB/s, done.


In [ ]:
# Make a dataframe using the 'pandas' module
import pandas as pd
dataframe = pd.read_excel('/content/product_dataset/updated_product_dataset.xlsx')
print(dataframe.head())

    Emotion                                               Text
0  Positive  close approximation red octane mat bought one ...
1   Neutral  little lumpy mat great foam padding itâ€™s use...
2  Positive  great pad love ddr not want metal pad get work...
3  Positive  excellent pad great product highly responsive ...
4  Positive  awesome great ddr pad works perfectly pc stepm...


In [ ]:
# Get unique emotions from the 'Emotion' column in the Dataset
dataframe["Emotion"].unique()

array(['Positive', 'Neutral', 'Negative'], dtype=object)

In [ ]:
# Add Labels to the dataset
encode_emotions = {"Neutral": 0, "Positive": 1, "Negative": 2}

In [ ]:
# replace the emotions with the Label
dataframe.replace(encode_emotions, inplace = True)
dataframe.head()

,Emotion,Text
0,1,close approximation red octane mat bought one ...
1,0,little lumpy mat great foam padding itâ€™s use...
2,1,great pad love ddr not want metal pad get work...
3,1,excellent pad great product highly responsive ...
4,1,awesome great ddr pad works perfectly pc stepm...


In [ ]:
# Convert the dataframe into list for easy processing
training_sentences = []
training_labels = []

for i in range(len(dataframe)):
  sentence = dataframe.loc[i, "Text"]
  training_sentences.append(sentence)
  label = dataframe.loc[i, "Emotion"]
  training_labels.append(label)

# printing 10th element
training_sentences[10], training_labels[10]


('arrived early included blank case wont able test game get switch tried brothers device recognized card smash digital download already arrived ahead schedule well packed even blank case wasnt expectingread full review',
 1)

In [ ]:
# Tokenize and pad your data
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
# Converting to numpy array
import numpy as np

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

In [ ]:
# import the necessary modules from tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Conv1D, Dropout, MaxPooling1D

# Create the model

model = tf.keras.Sequential([

    Embedding(vocab_size, embedding_dim, input_length = max_length),
    Dropout(0.2),
    Conv1D(filters = 256, kernel_size = 3, activation = 'relu'),
    MaxPooling1D(pool_size = 3),
    Conv1D(filters = 128, kernel_size = 3, activation = 'relu'),
    MaxPooling1D(pool_size = 3),
    LSTM(128),
    Dense(128, activation = 'relu'),
    Dropout(0.2),
    Dense(64, activation = 'relu'),
    Dense(6,activation = 'softmax')

])

# Compile the model

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# Get model Summary before training
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 dropout (Dropout)           (None, 100, 16)           0         
                                                                 
 conv1d (Conv1D)             (None, 98, 256)           12544     
                                                                 
 max_pooling1d (MaxPooling1  (None, 32, 256)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 30, 128)           98432     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 10, 128)           0         
 g1D)                                                   

In [ ]:
# Train your Model
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs = num_epochs, verbose = 2)

Epoch 1/30
671/671 - 23s - loss: 0.3546 - accuracy: 0.9224 - 23s/epoch - 35ms/step
Epoch 2/30
671/671 - 20s - loss: 0.1988 - accuracy: 0.9376 - 20s/epoch - 30ms/step
Epoch 3/30
671/671 - 19s - loss: 0.1453 - accuracy: 0.9507 - 19s/epoch - 28ms/step
Epoch 4/30
671/671 - 20s - loss: 0.1146 - accuracy: 0.9610 - 20s/epoch - 29ms/step
Epoch 5/30
671/671 - 19s - loss: 0.0985 - accuracy: 0.9650 - 19s/epoch - 28ms/step
Epoch 6/30
671/671 - 20s - loss: 0.0803 - accuracy: 0.9713 - 20s/epoch - 29ms/step
Epoch 7/30
671/671 - 19s - loss: 0.0687 - accuracy: 0.9765 - 19s/epoch - 28ms/step
Epoch 8/30
671/671 - 20s - loss: 0.0597 - accuracy: 0.9792 - 20s/epoch - 29ms/step
Epoch 9/30
671/671 - 19s - loss: 0.0523 - accuracy: 0.9838 - 19s/epoch - 28ms/step
Epoch 10/30
671/671 - 20s - loss: 0.0486 - accuracy: 0.9837 - 20s/epoch - 29ms/step
Epoch 11/30
671/671 - 19s - loss: 0.0437 - accuracy: 0.9857 - 19s/epoch - 28ms/step
Epoch 12/30
671/671 - 20s - loss: 0.0405 - accuracy: 0.9877 - 20s/epoch - 29ms/step
E

In [ ]:
# Save your Model

model.save("text_emotionchecker.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Test your model to make predictions

sentences = ["Great phone. Long lasting battery and awsome asthetics","Worst Headphones ever. Bluetooth is also not working","You can try for better camera options as well, but for this amount, its great to buy"]

sequences = tokenizer.texts_to_sequences(sentences)

padded = pad_sequences(sequences,maxlen = max_length, padding = padding_type, truncating = trunc_type)

result = model.predict(padded)

predict_class = np.argmax(result,axis = 1)

print(predict_class)

1/1 [==============================] - 1s 546ms/step
[1 2 0]
